# An interactive demonstration of image filtering with Discrete Fourier Transform

Based on OpenCV and xwidges running in Jupyter Notebook with xeus-cling C++ 14 kernel

In [1]:
#include <jupyter/opencv>
#include <opencv2/core.hpp>
#include <opencv2/imgproc.hpp>
#include "HeplerFuncs.hpp"

using namespace cv;

In [2]:
inline void displayImages(const Mat& img1, const Mat& img2) {
    const int border = 10;
    Size canvasSize(img1.cols + img2.cols + border, max(img1.rows, img2.rows));
    cv::Mat canvas(canvasSize, CV_8UC3);
    canvas.setTo(Scalar::all(255));

    cv::Mat rgbImage = makeRGB(img1);
    rgbImage.copyTo(canvas(Rect(0, 0, img1.cols, img1.rows)));
    rgbImage = makeRGB(img2);

    rgbImage.copyTo(canvas(Rect(img1.cols + border, 0, img2.cols, img2.rows)));
    im::mat_show(canvas);
}

In [3]:
// Load and prepare the image that will be filtered
const int imageSize = 512;

Mat hummingbirdImage = imread("hb_1024.png", IMREAD_GRAYSCALE);
hummingbirdImage.convertTo(hummingbirdImage, CV_32FC1, 1.0 / 255.0);
resize(hummingbirdImage, hummingbirdImage, Size(imageSize, imageSize));

# Discrete Fourier Transform
### $$ f(k) = \sum_{n=0}^{N-1}{s(n)\cdot e^{-ikn\frac{2\pi}{N}}} = \sum_{n=0}^{N-1}{s(n)[cos(kn{\frac{2\pi}{N}}) -i\cdot sin(kn{\frac{2\pi}{N}})]}$$
# 2D Discrete Fourier Transform
### $$ f(k, l) = \sum_{m=0}^{M-1}\sum_{n=0}^{N-1}{s(m, n)\cdot e^{-i2\pi(\frac{km}{M}+\frac{ln}{N})}} $$
### $$ f(k, l) =  \sum_{m=0}^{M-1}\sum_{n=0}^{N-1}{s(m, n)[cos(2\pi(\frac{km}{M}+\frac{ln}{N})) -i\cdot sin(2\pi(\frac{km}{M}+\frac{ln}{N}))]}$$

# The image filtering will be performed in the frequency domain
### $$ filteredImage = iDFT2D(element\_wise\_multiply(DFT2D(image), filter))$$

In [4]:
inline cv::Mat dft2dFilter(const cv::Mat& src, const cv::Mat& filter) {
    CV_Assert(src.size() == filter.size());
    CV_Assert_2(isPowerOf2(src.cols), isPowerOf2(filter.rows));
    CV_Assert_2(src.type() == CV_32F, filter.type() == CV_32F);

    cv::Mat matWithZeroes = cv::Mat::zeros(src.size(), CV_32F);
    cv::Mat complexInput1, complexOutput1;
    {
        cv::Mat planes[] = { src,  matWithZeroes };
        merge(planes, 2, complexInput1);
    }
    dft(complexInput1, complexOutput1, cv::DFT_SCALE | cv::DFT_COMPLEX_OUTPUT | cv::DFT_COMPLEX_INPUT);

    cv::Mat complexFilter;
    {
        cv::Mat planes[] = { filter,  filter };
        merge(planes, 2, complexFilter);
    }

    shiftFromCornerToCenter(complexFilter);

    cv::Mat frequencyDomainResult(src.size(), CV_32FC2);
    mulSpectrums(complexOutput1, complexFilter, frequencyDomainResult, 0);

    cv::Mat finalImage;
    idft(frequencyDomainResult, finalImage, cv::DFT_COMPLEX_OUTPUT);

    // Calculate the magnitude
    {
        cv::Mat planes[] = { cv::Mat(finalImage.size(), CV_32F), cv::Mat(finalImage.size(), CV_32F) };
        split(finalImage, planes);                   // planes[0] = Re(DFT(I), planes[1] = Im(DFT(I))
        magnitude(planes[0], planes[1], finalImage); // finalImage = magnitude
    }
    
    return finalImage;
}


## Using xwidgets to create some convenient UI

In [5]:
#include "xwidgets/xslider.hpp"
#include "xwidgets/xbutton.hpp"
#include "xwidgets/xcheckbox.hpp"
#include "xwidgets/xoutput.hpp"
#include "xwidgets/xbox.hpp"

auto radiusSlider = xw::slider<float>::initialize().value(16).min(1).max(100).step(1).description("Radius").finalize();
xw::button updateButton;
updateButton.description = "Update";
xw::checkbox invertCheckbox;
invertCheckbox.description = "Invert the filter";
invertCheckbox.value = false;
xw::hbox box;
box.add(radiusSlider);
box.add(invertCheckbox);
xw::output out;

In [6]:
#include <xcpp/xdisplay.hpp>

{
    auto func = [&](){
        // Using a scope guard to enable output capture
        auto g = out.guard();
        // Clear the previous output
        xcpp::clear_output();

        Mat filter = Mat::zeros(hummingbirdImage.size(), CV_32F);
        double radius = radiusSlider.value();
        
        circle(filter, Point(filter.cols/2, filter.rows/2), radius, Scalar::all(1.0), -1, 8, 0);
        if (invertCheckbox.value()) {
            filter = 1.0 - filter;
        }
        Mat filteredImage = dft2dFilter(hummingbirdImage, filter);
        displayImages(filter, filteredImage);
        
        //xcpp::display(radiusSlider);
        //xcpp::display(invertCheckbox);
        xcpp::display(box);
        xcpp::display(updateButton);
    };
    
    updateButton.on_click(func);
    
    func();
}

xcpp::display(out);

A Jupyter widget with unique id: f697bde81a5f4225ab887254204dafcb

Select the range of frequencies to be kept and click **Update**